Imports

In [15]:
from pyspark.sql import SparkSession
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql.window import Window #for ranking
from pyspark.sql.functions import lit, mean, stddev_pop
from pyspark.sql.functions import collect_set, collect_list
from pyspark.sql.functions import struct
from pyspark.sql.functions import slice
from pyspark.sql.functions import col
from pyspark.sql.functions import desc
from pyspark.sql.functions import udf
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql.types import DecimalType, ArrayType, IntegerType, FloatType
import pyspark.sql.functions as F
from pyspark.sql.functions import avg, broadcast, when

Define cosine similarity and weighted avg functions

In [23]:
# cosine similarity function
def cosine_similarity_udf(a, b):
    dot_product = sum([x * y for x, y in zip(a, b)])
    norm_a = sum([x**2 for x in a])**0.5
    norm_b = sum([x**2 for x in b])**0.5
    return dot_product / (norm_a * norm_b)


def weighted_avg_features(ratings, combined_vectors):
    if not ratings or not combined_vectors:
        return []

    weighted_sum = [0] * len(combined_vectors[0])
    total_weight = 0

    for rating, combined_vector in zip(ratings, combined_vectors):
        weight = float(rating)
        total_weight += weight
        weighted_sum = [ws + weight * f for ws, f in zip(weighted_sum, combined_vector)]

    if total_weight == 0:
        return weighted_sum

    return [ws / total_weight for ws in weighted_sum]

Load Dataset

In [17]:
spark = SparkSession.builder.appName('ReadMySQL') \
.config("spark.driver.memory", "32g") \
.config("spark.sql.pivotMaxValues", "1000000") \
.config("spark.jars", "C:\\Program Files (x86)\\MySQL\\Connector J 8.0\\mysql-connector-j-8.0.32.jar") \
.getOrCreate()

# sql = "select * from 01_sampled_games_2v2 WHERE playtime_forever IS NOT NULL AND playtime_forever > 0"
sql = sql = """
SELECT p.steamid, p.appid, p.playtime_2weeks, p.playtime_forever, p.dateretrieved, g.genre, d.Developer, pb.Publisher
FROM 01_sampled_games_2v2 AS p
JOIN games_genres AS g ON p.appid = g.appid
JOIN games_developers AS d ON p.appid = d.appid
JOIN games_publishers AS pb ON p.appid = pb.appid
WHERE p.playtime_forever IS NOT NULL AND p.playtime_forever > 0
"""
database = "steam"
user = "root"
password = "root"
server = "127.0.0.1"
port = 3307
jdbc_url = f"jdbc:mysql://{server}:{port}/{database}"
jdbc_driver = "com.mysql.cj.jdbc.Driver"

# Create a data frame by reading data from Oracle via JDBC
df = spark.read.format("jdbc") \
    .option("url", jdbc_url) \
    .option("query", sql) \
    .option("user", user) \
    .option("password", password) \
    .option("driver", jdbc_driver) \
    .load()

df = df.drop("playtime_2weeks", "dateretrieved")

In [18]:
# count number of rows in the dataframe
row_count = df.count()
# print the row count
print("Dataframe has", row_count, " rows.")
df.show()

Dataframe has 212589  rows.
+-----------------+-----+----------------+----------+--------------------+--------------------+
|          steamid|appid|playtime_forever|     genre|           Developer|           Publisher|
+-----------------+-----+----------------+----------+--------------------+--------------------+
|76561197960268000|  300|             109|    Action|               Valve|               Valve|
|76561197960268000| 1300|              94|    Action|Ritual Entertainment|Ritual Entertainment|
|76561197960268000| 2100|             110|    Action|      Arkane Studios|             Ubisoft|
|76561197960268000| 2100|             110|       RPG|      Arkane Studios|             Ubisoft|
|76561197960268000| 4000|             152|     Indie|   Facepunch Studios|               Valve|
|76561197960268000| 4000|             152|Simulation|   Facepunch Studios|               Valve|
|76561197960268000| 2600|              59|    Action|        Troika Games|          Activision|
|76561197960

Build item profiles

In [19]:
# build the item profiles
# Group the data by 'appid' and collect the genres for each game into a list
games_genres_df = df.groupBy("appid").agg(collect_set("genre").alias("genres"))
# Group the data by 'appid' and collect the developers for each game into a list
games_developers_df = df.groupBy("appid").agg(collect_set("Developer").alias("developers"))
# Group the data by 'appid' and collect the publishers for each game into a list
games_publishers_df = df.groupBy("appid").agg(collect_set("Publisher").alias("publishers"))

# Create a list of unique genres
unique_genres = sorted(df.select("genre").distinct().rdd.flatMap(lambda x: x).collect())
# Create a list of unique developers
unique_developers = sorted(df.select("Developer").distinct().rdd.flatMap(lambda x: x).collect())
# Create a list of unique publishers
unique_publishers = sorted(df.select("Publisher").distinct().rdd.flatMap(lambda x: x).collect())


# Define a UDF to create a binary vector for each game's genres
@udf(returnType=ArrayType(IntegerType()))
def genre_vector(genres):
    return [1 if genre in genres else 0 for genre in unique_genres]

# Define a UDF to create a binary vector for each game's developer
@udf(returnType=ArrayType(IntegerType()))
def developer_vector(developers):
    return [1 if developer in developers else 0 for developer in unique_developers]

# Define a UDF to create a binary vector for each game's publisher
@udf(returnType=ArrayType(IntegerType()))
def publisher_vector(publishers):
    return [1 if publisher in publishers else 0 for publisher in unique_publishers]


# Add a new column 'genre_vector' to the DataFrame
# the genre vector will now have a 1 for each genre that the game belongs to
games_genres_df = games_genres_df.withColumn("genre_vector", genre_vector("genres"))
# Add a new column 'developer_vector' to the DataFrame
games_developers_df = games_developers_df.withColumn("developer_vector", developer_vector("developers"))
# Add a new column 'publisher_vector' to the DataFrame
games_publishers_df = games_publishers_df.withColumn("publisher_vector", publisher_vector("publishers"))

# games_genres_df.show(truncate=False)
# Join the main DataFrame with the games_genres_df on appid to include the genre_vector
df = df.join(broadcast(games_genres_df.select("appid", "genre_vector")), on="appid")
# Join the main DataFrame with the games_developers_df
df = df.join(broadcast(games_developers_df.select("appid", "developer_vector")), on="appid")
# Join the main DataFrame with the games_publishers_df
df = df.join(broadcast(games_publishers_df.select("appid", "publisher_vector")), on="appid")

df.show()

+-----+-----------------+----------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+
|appid|          steamid|playtime_forever|     genre|           Developer|           Publisher|        genre_vector|    developer_vector|    publisher_vector|
+-----+-----------------+----------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  300|76561197960268000|             109|    Action|               Valve|               Valve|[1, 0, 0, 0, 0, 0...|[0, 0, 0, 0, 0, 0...|[0, 0, 0, 0, 0, 0...|
| 1300|76561197960268000|              94|    Action|Ritual Entertainment|Ritual Entertainment|[1, 0, 0, 0, 0, 0...|[0, 0, 0, 0, 0, 0...|[0, 0, 0, 0, 0, 0...|
| 2100|76561197960268000|             110|    Action|      Arkane Studios|             Ubisoft|[1, 0, 0, 0, 0, 0...|[0, 0, 0, 0, 0, 0...|[0, 0, 0, 0, 0, 0...|
| 2100|76561197960268000|             110|    

Build the user profile

In [20]:
# Calculate the per-game mean and standard deviation of the playtime column
game_stats = df.filter(col("playtime_forever") > 0).groupBy("appid").agg(
    mean("playtime_forever").alias("game_mean_playtime"),
    stddev_pop("playtime_forever").alias("game_stddev_playtime")
)

# Join the game_stats dataframe with the main dataframe
df = df.join(game_stats, "appid")

# Calculate the cut points and assign ratings
df = df.withColumn("cut_point_1", when(col("game_mean_playtime") - (col("game_stddev_playtime") * 0.5) > 0, col("game_mean_playtime") - (col("game_stddev_playtime") * 0.5)).otherwise(0))
df = df.withColumn("cut_point_2", col("game_mean_playtime"))
df = df.withColumn("cut_point_3", col("game_mean_playtime") + (col("game_stddev_playtime") * 0.5))
df = df.withColumn("cut_point_4", col("game_mean_playtime") + col("game_stddev_playtime"))

df = df.withColumn(
    "ratings",
    when(col("playtime_forever") <= col("cut_point_1"), lit(1))
    .when((col("playtime_forever") > col("cut_point_1")) & (col("playtime_forever") <= col("cut_point_2")), lit(2))
    .when((col("playtime_forever") > col("cut_point_2")) & (col("playtime_forever") <= col("cut_point_3")), lit(3))
    .when((col("playtime_forever") > col("cut_point_3")) & (col("playtime_forever") <= col("cut_point_4")), lit(4))
    .otherwise(lit(5))
)

# Update the user profile calculation to use the new ratings column
user_aggregated_data = df.groupBy("steamid").agg(
    collect_list("genre_vector").alias("genres_list"),
    collect_list("ratings").alias("ratings_list")
)
# Show df with new changes(without genres)
# Drop the genre_vector and genre columns from the DataFrame
df_without_info = df.drop("genre_vector", "genre", "developer_vector", "Developer", "publisher_vector", "Publisher")

# Show the DataFrame without the genre_vector and genre columns
df_without_info.show()
#df.show()

+------+-----------------+----------------+------------------+--------------------+-----------------+-----------+------------------+-----------------+-------+
| appid|          steamid|playtime_forever|game_mean_playtime|game_stddev_playtime|      cut_point_1|cut_point_2|       cut_point_3|      cut_point_4|ratings|
+------+-----------------+----------------+------------------+--------------------+-----------------+-----------+------------------+-----------------+-------+
|285900|76561198034777000|               3|             102.6|   77.31649241914691|63.94175379042654|      102.6|141.25824620957346|179.9164924191469|      1|
|285900|76561198034777000|               3|             102.6|   77.31649241914691|63.94175379042654|      102.6|141.25824620957346|179.9164924191469|      1|
|285900|76561198034777000|               3|             102.6|   77.31649241914691|63.94175379042654|      102.6|141.25824620957346|179.9164924191469|      1|
|285900|76561198034777000|               3|   

Build user profile off this

In [24]:
# combine vectors

# Define a UDF to combine the vectors
@udf(returnType=ArrayType(IntegerType()))
def combined_vector(genre_vector, developer_vector, publisher_vector):
    return genre_vector + developer_vector + publisher_vector

# Add 'combined_vector' to the DataFrame
df = df.withColumn("combined_vector", combined_vector("genre_vector", "developer_vector", "publisher_vector"))

# Update the user profile calculation to use the new ratings column and include developer and publisher vectors
user_aggregated_data = df.groupBy("steamid").agg(
    collect_list("combined_vector").alias("combined_vectors_list"),  
    collect_list("ratings").alias("ratings_list")
)

# Define a UDF to calculate the weighted average of genre vectors
weighted_avg_features_udf = udf(weighted_avg_features, ArrayType(FloatType()))

# Calculate the user profile as the weighted average of rated item profiles (combined genre, developer, and publisher vectors)
user_profiles = user_aggregated_data.withColumn("user_profile", weighted_avg_features_udf("ratings_list", "combined_vectors_list"))

user_profiles.show()

+-----------------+---------------------+--------------------+--------------------+
|          steamid|combined_vectors_list|        ratings_list|        user_profile|
+-----------------+---------------------+--------------------+--------------------+
|76561197960271000| [[1, 0, 0, 0, 0, ...|                 [2]|[1.0, 0.0, 0.0, 0...|
|76561197960306000| [[1, 0, 0, 0, 0, ...|[2, 2, 2, 2, 2, 2...|[0.6334587, 0.257...|
|76561197960314000| [[1, 0, 0, 0, 0, ...|              [2, 2]|[1.0, 0.0, 0.0, 0...|
|76561197960334000| [[1, 0, 0, 0, 0, ...|[4, 2, 2, 2, 2, 1...|[1.0, 0.2, 0.0, 0...|
|76561197960338000| [[1, 0, 0, 0, 0, ...|                 [2]|[1.0, 0.0, 0.0, 0...|
|76561197960342000| [[1, 0, 0, 0, 0, ...|[2, 2, 2, 2, 2, 2...|[1.0, 0.5555556, ...|
|76561197960346000| [[1, 0, 0, 0, 0, ...|                 [2]|[1.0, 0.0, 0.0, 0...|
|76561197960394000| [[0, 0, 0, 0, 0, ...|              [2, 2]|[0.0, 0.0, 0.0, 0...|
|76561197960421000| [[0, 0, 0, 0, 0, ...|[3, 2, 2, 2, 2, 2...|[0.29807693, 0

Cosine similarity

In [27]:
# prediction heuristics
# calculate cosine distance of an item and user profile

# 1. create udf for cosine similarity
cosine_similarity = udf(cosine_similarity_udf, FloatType())
# create dataframe with combined vectors
games_combined_vectors_df = df.select("appid", "combined_vector").distinct()
# cross join the the combined vectors with the user_profiles
cross_joined = games_combined_vectors_df.crossJoin(user_profiles)

# calculate the cosine similarity between each item and user
recommendations = cross_joined.withColumn(
    "similarity", cosine_similarity("combined_vector", "user_profile")
)

# sort based on similarity score
sorted_recommendations = recommendations.sort(desc("similarity"))

sorted_recommendations.show(10)

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

recommender

In [25]:
# Create a window by steamid and similarity to get ranking
window_spec = Window.partitionBy("steamid").orderBy(desc("similarity"))

ranked_recommendations = sorted_recommendations.withColumn("rank", F.row_number().over(window_spec))

top_10_recommendations = ranked_recommendations.filter(ranked_recommendations.rank <= 10)
top_10_recommendations.show(10)

+------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+----------+----+
| appid|              genres|        genre_vector|          steamid|         genres_list|        ratings_list|        user_profile|similarity|rank|
+------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+----------+----+
|  2400|[RPG, Action, Indie]|[1, 0, 0, 0, 0, 0...|76561197960334000|[[1, 0, 0, 0, 0, ...|[4, 2, 2, 2, 2, 1...|[1.0, 0.2, 0.0, 0...|0.92450035|   1|
| 33100|[RPG, Action, Indie]|[1, 0, 0, 0, 0, 0...|76561197960334000|[[1, 0, 0, 0, 0, ...|[4, 2, 2, 2, 2, 1...|[1.0, 0.2, 0.0, 0...|0.92450035|   2|
| 37100|[RPG, Action, Indie]|[1, 0, 0, 0, 0, 0...|76561197960334000|[[1, 0, 0, 0, 0, ...|[4, 2, 2, 2, 2, 1...|[1.0, 0.2, 0.0, 0...|0.92450035|   3|
| 91700|[RPG, Action, Indie]|[1, 0, 0, 0, 0, 0...|76561197960334000|[[1, 0, 0, 0, 0, ...|[4, 2, 2, 2, 2, 1...|[1